# **Chapter 25: Native iOS Integration**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Embed Flutter within existing iOS UIViewControllers using FlutterEngine and FlutterViewController
- Implement bidirectional communication between Swift/Objective-C and Dart using MethodChannel and EventChannel
- Configure Info.plist for permissions, privacy descriptions, and app capabilities
- Implement Universal Links and custom URL schemes for deep linking
- Handle iOS app lifecycle states (foreground, background, suspended) with proper resource management
- Implement background fetch and background processing tasks using iOS BGTaskScheduler
- Manage iOS-specific memory constraints and optimize for low-memory scenarios

---

## **Prerequisites**

- Completed Chapter 23: Platform Integration (MethodChannel, EventChannel fundamentals)
- Completed Chapter 24: Native Android Integration (understanding of platform embedding concepts)
- Intermediate knowledge of iOS development (UIViewController lifecycle, Swift/Objective-C)
- Understanding of Swift concurrency (async/await, completion handlers)
- Familiarity with Xcode, CocoaPods, and iOS provisioning profiles
- Access to Apple Developer account for testing Universal Links

---

## **25.1 UIViewController & FlutterEngine Embedding**

iOS embedding architecture differs from Android in that `FlutterViewController` manages the engine lifecycle, and you can pre-warm engines for faster startup.

### **Standard FlutterViewController Setup**

```swift
// ios/Runner/AppDelegate.swift
// Standard configuration with MethodChannel registration

import UIKit
import Flutter

@main
@objc class AppDelegate: FlutterAppDelegate {
    
    // Lazy initialization of FlutterEngine for pre-warming
    lazy var flutterEngine = FlutterEngine(name: "my_engine_id")
    
    override func application(
        _ application: UIApplication,
        didFinishLaunchingWithOptions launchOptions: [UIApplication.LaunchOptionsKey: Any]?
    ) -> Bool {
        
        // Pre-warm engine before showing UI (optional optimization)
        flutterEngine.run()
        
        // Register plugins with pre-warmed engine
        GeneratedPluginRegistrant.register(with: flutterEngine)
        
        // Setup platform channels before root view controller is created
        setupMethodChannels()
        
        return super.application(application, didFinishLaunchingWithOptions: launchOptions)
    }
    
    private func setupMethodChannels() {
        let batteryChannel = FlutterMethodChannel(
            name: "com.example.app/battery",
            binaryMessenger: flutterEngine.binaryMessenger
        )
        
        batteryChannel.setMethodCallHandler { [weak self] (call, result) in
            // Avoid retain cycles with [weak self]
            self?.handleBatteryMethod(call, result: result)
        }
    }
    
    private func handleBatteryMethod(_ call: FlutterMethodCall, result: @escaping FlutterResult) {
        switch call.method {
        case "getBatteryLevel":
            let device = UIDevice.current
            device.isBatteryMonitoringEnabled = true
            
            if device.batteryState == .unknown {
                result(FlutterError(
                    code: "UNAVAILABLE",
                    message: "Battery info unavailable",
                    details: nil
                ))
            } else {
                // batteryLevel returns 0.0 to 1.0
                result(Int(device.batteryLevel * 100))
            }
            
        case "getDeviceInfo":
            let info = [
                "systemName": UIDevice.current.systemName,
                "systemVersion": UIDevice.current.systemVersion,
                "model": UIDevice.current.model,
                "localizedModel": UIDevice.current.localizedModel
            ]
            result(info)
            
        default:
            result(FlutterMethodNotImplemented)
        }
    }
    
    // Memory warning handling - crucial for iOS
    override func applicationDidReceiveMemoryWarning(_ application: UIApplication) {
        // Notify Flutter to clear caches
        let channel = FlutterMethodChannel(
            name: "com.example.app/lifecycle",
            binaryMessenger: flutterEngine.binaryMessenger
        )
        channel.invokeMethod("memoryWarning", arguments: nil)
    }
}
```

**Explanation:**

- **FlutterEngine Pre-warming**: Unlike Android where you cache in `Application`, on iOS you typically create the engine in `AppDelegate` before `GeneratedPluginRegistrant.register()`. This reduces cold start time from ~2 seconds to ~500ms.
- **Binary Messenger**: The communication bridge. Must be obtained from the engine, not the view controller, when pre-warming.
- **[weak self]**: Critical in Swift to prevent retain cycles between the channel handler and AppDelegate. Without this, the AppDelegate leaks.
- **Battery Monitoring**: iOS requires explicitly enabling monitoring before accessing battery state. This prompts the user with a permission dialog on first access.
- **Memory Warnings**: iOS aggressively terminates apps using too much memory. Override `applicationDidReceiveMemoryWarning` to notify Flutter to drop image caches, purge temporary files, and reduce memory footprint.

### **Embedding Flutter in Existing View Controllers**

```swift
// ios/Runner/NativeHostViewController.swift
// Hosting Flutter as a child view controller in native iOS UI

import UIKit
import Flutter

class NativeHostViewController: UIViewController {
    
    // Flutter engine instance - can be shared or dedicated
    var flutterEngine: FlutterEngine?
    
    // Container for Flutter UI
    var flutterContainerView: UIView!
    
    // Native UI elements
    var nativeToolbar: UIToolbar!
    var showFlutterButton: UIButton!
    
    override func viewDidLoad() {
        super.viewDidLoad()
        setupNativeUI()
        setupFlutterEngine()
    }
    
    private func setupNativeUI() {
        view.backgroundColor = .systemBackground
        
        // Native toolbar
        nativeToolbar = UIToolbar(frame: CGRect.zero)
        nativeToolbar.translatesAutoresizingMaskIntoConstraints = false
        view.addSubview(nativeToolbar)
        
        // Button to toggle Flutter visibility
        showFlutterButton = UIButton(type: .system)
        showFlutterButton.setTitle("Show Flutter", for: .normal)
        showFlutterButton.addTarget(self, action: #selector(toggleFlutter), for: .touchUpInside)
        showFlutterButton.translatesAutoresizingMaskIntoConstraints = false
        view.addSubview(showFlutterButton)
        
        // Container for Flutter
        flutterContainerView = UIView()
        flutterContainerView.translatesAutoresizingMaskIntoConstraints = false
        flutterContainerView.isHidden = true
        view.addSubview(flutterContainerView)
        
        // Auto Layout constraints
        NSLayoutConstraint.activate([
            nativeToolbar.topAnchor.constraint(equalTo: view.safeAreaLayoutGuide.topAnchor),
            nativeToolbar.leadingAnchor.constraint(equalTo: view.leadingAnchor),
            nativeToolbar.trailingAnchor.constraint(equalTo: view.trailingAnchor),
            
            showFlutterButton.topAnchor.constraint(equalTo: nativeToolbar.bottomAnchor, constant: 20),
            showFlutterButton.centerXAnchor.constraint(equalTo: view.centerXAnchor),
            
            flutterContainerView.topAnchor.constraint(equalTo: showFlutterButton.bottomAnchor, constant: 20),
            flutterContainerView.leadingAnchor.constraint(equalTo: view.leadingAnchor),
            flutterContainerView.trailingAnchor.constraint(equalTo: view.trailingAnchor),
            flutterContainerView.bottomAnchor.constraint(equalTo: view.bottomAnchor)
        ])
    }
    
    private func setupFlutterEngine() {
        // Option 1: Use cached engine from AppDelegate
        if let appDelegate = UIApplication.shared.delegate as? AppDelegate {
            flutterEngine = appDelegate.flutterEngine
        }
        
        // Option 2: Create dedicated engine
        if flutterEngine == nil {
            flutterEngine = FlutterEngine(name: "host_engine")
            flutterEngine?.run()
            GeneratedPluginRegistrant.register(with: flutterEngine!)
        }
    }
    
    @objc private func toggleFlutter() {
        if flutterContainerView.isHidden {
            embedFlutterView()
            flutterContainerView.isHidden = false
            showFlutterButton.setTitle("Hide Flutter", for: .normal)
        } else {
            flutterContainerView.isHidden = true
            detachFlutterView()
            showFlutterButton.setTitle("Show Flutter", for: .normal)
        }
    }
    
    private func embedFlutterView() {
        guard let engine = flutterEngine else { return }
        
        // Create FlutterViewController with existing engine
        let flutterViewController = FlutterViewController(
            engine: engine,
            nibName: nil,
            bundle: nil
        )
        
        // Add as child view controller (critical for lifecycle)
        addChild(flutterViewController)
        
        // Add view to container
        flutterViewController.view.frame = flutterContainerView.bounds
        flutterViewController.view.autoresizingMask = [.flexibleWidth, .flexibleHeight]
        flutterContainerView.addSubview(flutterViewController.view)
        
        // Notify child it was added
        flutterViewController.didMove(toParent: self)
        
        // Setup method channel for this specific instance
        setupInstanceChannel(for: flutterViewController)
    }
    
    private func detachFlutterView() {
        // Find and remove FlutterViewController
        for child in children {
            if let flutterVC = child as? FlutterViewController {
                // Notify it will be removed
                flutterVC.willMove(toParent: nil)
                // Remove view
                flutterVC.view.removeFromSuperview()
                // Remove from parent
                flutterVC.removeFromParent()
            }
        }
    }
    
    private func setupInstanceChannel(for viewController: FlutterViewController) {
        let channel = FlutterMethodChannel(
            name: "com.example.app/host_\(viewController.hashValue)",
            binaryMessenger: viewController.binaryMessenger
        )
        
        channel.setMethodCallHandler { [weak self] (call, result) in
            switch call.method {
            case "getNativeData":
                result(["platform": "iOS", "host": "NativeHostViewController"])
            case "closeFlutter":
                self?.toggleFlutter()
                result(nil)
            default:
                result(FlutterMethodNotImplemented)
            }
        }
    }
    
    // Handle device rotation
    override func viewWillTransition(to size: CGSize, with coordinator: UIViewControllerTransitionCoordinator) {
        super.viewWillTransition(to: size, with: coordinator)
        
        // Notify Flutter of rotation
        // This is handled automatically by FlutterViewController, but
        // you can add custom logic here if needed
    }
}
```

**Explanation:**

- **Child View Controller**: Adding Flutter as a child using `addChild()` and `didMove(toParent:)` is mandatory for proper view controller containment and lifecycle propagation (rotation, memory warnings, etc.).
- **Engine Sharing vs. Dedicated**: You can share one engine across multiple FlutterViewControllers (singleton pattern) or create dedicated engines per controller. Sharing uses less memory but couples all Flutter instances.
- **Autoresizing Mask**: `flexibleWidth` and `flexibleHeight` ensure Flutter view resizes automatically when the container bounds change (rotation, split-screen multitasking).
- **Hash-based Channels**: When embedding multiple Flutter instances, use unique channel names (e.g., incorporating `hashValue`) to communicate with specific instances.
- **View Removal**: Proper cleanup requires calling `willMove(toParent: nil)` before removing, and `removeFromParent()` after, to prevent view controller hierarchy corruption.

---

## **25.2 Swift Concurrency with Platform Channels**

Modern iOS development uses Swift's async/await pattern. Bridging this with Flutter's callback-based platform channels requires careful handling.

### **Async/Await Bridge Pattern**

```swift
// ios/Runner/Channels/AsyncMethodChannel.swift

import Flutter
import UIKit

/// Wrapper that bridges Swift async/await with Flutter platform channels
class AsyncMethodChannel {
    private let channel: FlutterMethodChannel
    
    init(messenger: FlutterBinaryMessenger, name: String) {
        channel = FlutterMethodChannel(name: name, binaryMessenger: messenger)
        setupHandler()
    }
    
    private func setupHandler() {
        channel.setMethodCallHandler { [weak self] call, result in
            // Convert callback-based Flutter to async Swift
            Task {
                await self?.handleMethodCall(call, result: result)
            }
        }
    }
    
    private func handleMethodCall(_ call: FlutterMethodCall, result: @escaping FlutterResult) async {
        switch call.method {
        case "fetchUserProfile":
            await handleFetchUser(call: call, result: result)
            
        case "uploadImage":
            await handleImageUpload(call: call, result: result)
            
        case "processData":
            // Run CPU-intensive work on background
            await handleProcessing(call: call, result: result)
            
        default:
            result(FlutterMethodNotImplemented)
        }
    }
    
    private func handleFetchUser(call: FlutterMethodCall, result: @escaping FlutterResult) async {
        guard let userId = call.arguments as? String else {
            result(FlutterError(code: "INVALID_ARG", message: "User ID required", details: nil))
            return
        }
        
        do {
            // Async network call
            let user = try await NetworkService.shared.fetchUser(id: userId)
            
            // Convert to Flutter-compatible dictionary
            let data: [String: Any] = [
                "id": user.id,
                "name": user.name,
                "email": user.email,
                "avatarUrl": user.avatarUrl?.absoluteString ?? NSNull(),
                "createdAt": ISO8601DateFormatter().string(from: user.createdAt)
            ]
            
            // Result must be called on main thread
            await MainActor.run {
                result(data)
            }
        } catch {
            await MainActor.run {
                result(FlutterError(code: "NETWORK_ERROR", message: error.localizedDescription, details: nil))
            }
        }
    }
    
    private func handleImageUpload(call: FlutterMethodCall, result: @escaping FlutterResult) async {
        guard let args = call.arguments as? [String: Any],
              let imagePath = args["path"] as? String else {
            result(FlutterError(code: "INVALID_ARG", message: "Image path required", details: nil))
            return
        }
        
        let quality = args["quality"] as? Int ?? 80
        
        // Background processing for image compression
        let processedData = await Task.detached(priority: .userInitiated) {
            return await ImageProcessor.compressImage(at: imagePath, quality: quality)
        }.value
        
        do {
            let uploadUrl = try await CloudStorage.upload(processedData)
            await MainActor.run {
                result(["url": uploadUrl, "size": processedData.count])
            }
        } catch {
            await MainActor.run {
                result(FlutterError(code: "UPLOAD_FAILED", message: error.localizedDescription, details: nil))
            }
        }
    }
    
    private func handleProcessing(call: FlutterMethodCall, result: @escaping FlutterResult) async {
        // Simulate heavy computation
        let output = await Task.detached(priority: .background) {
            var sum = 0
            for i in 0..<1_000_000 {
                sum += i
            }
            return sum
        }.value
        
        await MainActor.run {
            result(["result": output])
        }
    }
}

// Supporting services
class NetworkService {
    static let shared = NetworkService()
    
    func fetchUser(id: String) async throws -> User {
        // Simulate network delay
        try await Task.sleep(nanoseconds: 1_000_000_000) // 1 second
        
        return User(
            id: id,
            name: "John Doe",
            email: "john@example.com",
            avatarUrl: URL(string: "https://example.com/avatar.jpg"),
            createdAt: Date()
        )
    }
}

struct User {
    let id: String
    let name: String
    let email: String
    let avatarUrl: URL?
    let createdAt: Date
}

class ImageProcessor {
    static func compressImage(at path: String, quality: Int) async -> Data {
        // Image processing logic
        return Data()
    }
}

class CloudStorage {
    static func upload(_ data: Data) async throws -> String {
        return "https://cdn.example.com/image.jpg"
    }
}
```

**Explanation:**

- **Task**: Swift concurrency primitive. Wrapping the handler in `Task` allows using `await` inside the callback-based Flutter channel handler.
- **MainActor.run**: Flutter's `result` callback must be invoked on the main thread. `MainActor.run` ensures this even if the preceding async work was on a background actor.
- **Task.detached**: For CPU-intensive work (image processing, calculations), use `detached` with appropriate priority (`.userInitiated` for user-visible work, `.background` for maintenance tasks) to prevent blocking the main actor.
- **Error Handling**: Swift `throws` maps naturally to Flutter's `FlutterError`. Always catch errors and convert them to prevent Swift exceptions from crashing the app.
- **NSNull()**: Use `NSNull()` (not `nil`) for null values in dictionaries sent to Flutter, as `nil` values in NSDictionary entries can cause crashes or missing keys.

---

## **25.3 Info.plist Configuration**

iOS uses Info.plist for permissions, capabilities, and app metadata. Modern iOS requires explicit privacy descriptions for all sensitive APIs.

### **Info.plist Configuration**

```xml
<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
<dict>
    <!-- Bundle Configuration -->
    <key>CFBundleName</key>
    <string>My Flutter App</string>
    <key>CFBundleDisplayName</key>
    <string>FlutterApp</string>
    <key>CFBundleIdentifier</key>
    <string>com.example.app</string>
    <key>CFBundleVersion</key>
    <string>1.0.0</string>
    <key>CFBundleShortVersionString</key>
    <string>1.0</string>
    
    <!-- Scene Configuration (iOS 13+) -->
    <key>UIApplicationSceneManifest</key>
    <dict>
        <key>UIApplicationSupportsMultipleScenes</key>
        <false/>
        <key>UISceneConfigurations</key>
        <dict>
            <key>UIWindowSceneSessionRoleApplication</key>
            <array>
                <dict>
                    <key>UISceneConfigurationName</key>
                    <string>Default Configuration</string>
                    <key>UISceneDelegateClassName</key>
                    <string>$(PRODUCT_MODULE_NAME).SceneDelegate</string>
                </dict>
            </array>
        </dict>
    </dict>
    
    <!-- Privacy Descriptions (Required for App Store) -->
    <key>NSCameraUsageDescription</key>
    <string>This app needs camera access to scan QR codes and take profile photos.</string>
    
    <key>NSPhotoLibraryUsageDescription</key>
    <string>This app needs photo library access to let you select profile pictures.</string>
    
    <key>NSPhotoLibraryAddUsageDescription</key>
    <string>This app needs permission to save photos to your library.</string>
    
    <key>NSLocationWhenInUseUsageDescription</key>
    <string>This app needs location access to show nearby places.</string>
    
    <key>NSLocationAlwaysAndWhenInUseUsageDescription</key>
    <string>This app needs background location to track your runs even when closed.</string>
    
    <key>NSMicrophoneUsageDescription</key>
    <string>This app needs microphone access for voice messages.</string>
    
    <key>NSFaceIDUsageDescription</key>
    <string>This app uses Face ID to secure your data.</string>
    
    <key>NSUserTrackingUsageDescription</key>
    <string>This app tracks usage data to improve your experience.</string>
    
    <!-- Background Modes -->
    <key>UIBackgroundModes</key>
    <array>
        <string>fetch</string>
        <string>remote-notification</string>
        <string>processing</string>
        <string>location</string>
        <string>audio</string>
    </array>
    
    <!-- URL Types (Custom URL Schemes) -->
    <key>CFBundleURLTypes</key>
    <array>
        <dict>
            <key>CFBundleURLName</key>
            <string>com.example.app</string>
            <key>CFBundleURLSchemes</key>
            <array>
                <string>myapp</string>
            </array>
        </dict>
    </array>
    
    <!-- Associated Domains (Universal Links) -->
    <key>com.apple.developer.associated-domains</key>
    <array>
        <string>applinks:example.com</string>
        <string>applinks:*.example.com</string>
        <string>webcredentials:example.com</string>
    </array>
    
    <!-- Security -->
    <key>NSAppTransportSecurity</key>
    <dict>
        <key>NSAllowsArbitraryLoads</key>
        <false/>
        <key>NSExceptionDomains</key>
        <dict>
            <key>localhost</key>
            <dict>
                <key>NSExceptionAllowsInsecureHTTPLoads</key>
                <true/>
            </dict>
        </dict>
    </dict>
    
    <!-- Orientation Support -->
    <key>UISupportedInterfaceOrientations</key>
    <array>
        <string>UIInterfaceOrientationPortrait</string>
        <string>UIInterfaceOrientationLandscapeLeft</string>
        <string>UIInterfaceOrientationLandscapeRight</string>
    </array>
    
    <!-- Status Bar Style -->
    <key>UIViewControllerBasedStatusBarAppearance</key>
    <false/>
    
    <!-- Disable Hardware Keyboard (for simulators) -->
    <key>UIRequiresPersistentWiFi</key>
    <true/>
</dict>
</plist>
```

**Explanation:**

- **Privacy Descriptions**: Starting iOS 10+, Apple requires `NS*UsageDescription` keys for camera, photos, location, microphone, etc. The strings appear in system permission dialogs. Missing descriptions cause immediate App Store rejection.
- **SceneDelegate**: iOS 13+ uses SceneDelegate for multi-window support. Flutter apps typically set `UIApplicationSupportsMultipleScenes` to false unless supporting iPad multitasking.
- **UIBackgroundModes**: Declare background capabilities:
  - `fetch`: Background fetch for content updates
  - `remote-notification`: Silent push notifications
  - `processing`: BGProcessingTask for maintenance (iOS 13+)
  - `location`: Background location updates
  - `audio`: Background audio playback
- **CFBundleURLSchemes**: Custom URL schemes (e.g., `myapp://`). Must be unique to avoid conflicts with other apps.
- **Associated Domains**: Enable Universal Links (HTTPS deep links) and shared web credentials. Must match entitlements and Apple App Site Association file on your server.

---

## **25.4 Universal Links & URL Handling**

Universal Links allow your app to open when users tap links to your website, providing a seamless transition from web to app.

### **Universal Link Handling**

```swift
// ios/Runner/AppDelegate.swift
// Handling Universal Links and Custom URL Schemes

import UIKit
import Flutter

@main
@objc class AppDelegate: FlutterAppDelegate {
    
    // Store initial link for Flutter to retrieve
    private var initialLink: String?
    private var linkChannel: FlutterMethodChannel?
    
    override func application(
        _ application: UIApplication,
        didFinishLaunchingWithOptions launchOptions: [UIApplication.LaunchOptionsKey: Any]?
    ) -> Bool {
        
        // Check if launched from URL
        if let url = launchOptions?[.url] as? URL {
            initialLink = url.absoluteString
        }
        
        GeneratedPluginRegistrant.register(with: self)
        setupLinkChannel()
        
        return super.application(application, didFinishLaunchingWithOptions: launchOptions)
    }
    
    private func setupLinkChannel() {
        let controller = window?.rootViewController as! FlutterViewController
        linkChannel = FlutterMethodChannel(
            name: "com.example.app/links",
            binaryMessenger: controller.binaryMessenger
        )
        
        linkChannel?.setMethodCallHandler { [weak self] (call, result) in
            switch call.method {
            case "getInitialLink":
                result(self?.initialLink)
                // Clear after first read to prevent re-processing
                self?.initialLink = nil
            case "clearInitialLink":
                self?.initialLink = nil
                result(nil)
            default:
                result(FlutterMethodNotImplemented)
            }
        }
    }
    
    // Handle Universal Links (iOS 9+)
    override func application(
        _ application: UIApplication,
        continue userActivity: NSUserActivity,
        restorationHandler: @escaping ([UIUserActivityRestoring]?) -> Void
    ) -> Bool {
        
        // Check if it's a web link (Universal Link)
        if userActivity.activityType == NSUserActivityTypeBrowsingWeb,
           let url = userActivity.webpageURL {
            
            handleUniversalLink(url)
            return true
        }
        
        return false
    }
    
    // Handle Custom URL Schemes
    override func application(
        _ app: UIApplication,
        open url: URL,
        options: [UIApplication.OpenURLOptionsKey : Any] = [:]
    ) -> Bool {
        
        // Handle URL scheme (myapp://)
        if url.scheme == "myapp" {
            handleCustomScheme(url)
            return true
        }
        
        // Handle file sharing (open-in...)
        if url.isFileURL {
            handleSharedFile(url)
            return true
        }
        
        return false
    }
    
    private func handleUniversalLink(_ url: URL) {
        // Parse https://example.com/app/product/123
        let path = url.path
        let query = url.query
        
        let linkData: [String: Any] = [
            "type": "universal_link",
            "url": url.absoluteString,
            "path": path,
            "query": query ?? NSNull(),
            "host": url.host ?? NSNull()
        ]
        
        // If Flutter is ready, send immediately
        if let channel = linkChannel {
            channel.invokeMethod("onLink", arguments: linkData)
        } else {
            // Store for later retrieval
            initialLink = url.absoluteString
        }
    }
    
    private func handleCustomScheme(_ url: URL) {
        // Parse myapp://profile?id=123
        guard let components = URLComponents(url: url, resolvingAgainstBaseURL: true) else {
            return
        }
        
        let path = components.path  // "profile"
        let queryItems = components.queryItems ?? []
        let params = queryItems.reduce(into: [String: String]()) { result, item in
            result[item.name] = item.value
        }
        
        let linkData: [String: Any] = [
            "type": "custom_scheme",
            "scheme": url.scheme ?? "",
            "path": path,
            "params": params
        ]
        
        linkChannel?.invokeMethod("onLink", arguments: linkData)
    }
    
    private func handleSharedFile(_ url: URL) {
        // Security-scoped resource handling
        guard url.startAccessingSecurityScopedResource() else {
            return
        }
        
        defer { url.stopAccessingSecurityScopedResource() }
        
        do {
            // Copy to app sandbox if needed
            let fileData = try Data(contentsOf: url)
            let fileName = url.lastPathComponent
            
            linkChannel?.invokeMethod("onFileShared", arguments: [
                "fileName": fileName,
                "fileSize": fileData.count,
                "mimeType": mimeType(for: url)
            ])
        } catch {
            print("Error reading shared file: \(error)")
        }
    }
    
    private func mimeType(for url: URL) -> String {
        let ext = url.pathExtension.lowercased()
        switch ext {
        case "jpg", "jpeg": return "image/jpeg"
        case "png": return "image/png"
        case "pdf": return "application/pdf"
        default: return "application/octet-stream"
        }
    }
}
```

**Explanation:**

- **NSUserActivityTypeBrowsingWeb**: Indicates a Universal Link triggered the app. The URL is in `webpageURL`.
- **Security-Scoped Resources**: When receiving files from other apps (Open In...), iOS Sandboxing requires calling `startAccessingSecurityScopedResource()` before reading. Always call `stopAccessingSecurityScopedResource()` in a `defer` block to prevent permission leaks.
- **URLComponents**: Parses query strings into key-value pairs more reliably than string manipulation.
- **Multiple Entry Points**: iOS can launch your app via Universal Links (continue userActivity), Custom Schemes (open url), or Push Notifications. Each requires separate handling but should converge to the same Flutter channel.

### **Apple App Site Association (AASA)**

For Universal Links to work, host this file at `https://example.com/.well-known/apple-app-site-association`:

```json
{
  "applinks": {
    "apps": [],
    "details": [
      {
        "appID": "TEAM_ID.com.example.app",
        "paths": ["/app/*", "/product/*", "/user/profile"],
        "components": [
          {
            "/": "/app/*",
            "comment": "Matches any URL with path starting with /app/"
          }
        ]
      }
    ]
  },
  "webcredentials": {
    "apps": ["TEAM_ID.com.example.app"]
  }
}
```

---

## **25.5 Background Processing**

iOS provides Background Fetch (deprecated) and BGTaskScheduler (iOS 13+) for background work. iOS is much more restrictive than Android regarding background execution.

### **Background Task Implementation**

```swift
// ios/Runner/AppDelegate.swift
// Background fetch and processing tasks

import UIKit
import Flutter
import BackgroundTasks  // iOS 13+

@main
@objc class AppDelegate: FlutterAppDelegate {
    
    // Task identifiers (must match Info.plist BGTaskSchedulerPermittedIdentifiers)
    private let refreshTaskIdentifier = "com.example.app.refresh"
    private let processingTaskIdentifier = "com.example.app.processing"
    
    override func application(
        _ application: UIApplication,
        didFinishLaunchingWithOptions launchOptions: [UIApplication.LaunchOptionsKey: Any]?
    ) -> Bool {
        
        GeneratedPluginRegistrant.register(with: self)
        
        // Register background tasks (iOS 13+)
        if #available(iOS 13.0, *) {
            BGTaskScheduler.shared.register(
                forTaskWithIdentifier: refreshTaskIdentifier,
                using: nil
            ) { [weak self] task in
                self?.handleAppRefresh(task: task as! BGAppRefreshTask)
            }
            
            BGTaskScheduler.shared.register(
                forTaskWithIdentifier: processingTaskIdentifier,
                using: nil
            ) { [weak self] task in
                self?.handleProcessingTask(task: task as! BGProcessingTask)
            }
        }
        
        // Schedule initial tasks
        scheduleAppRefresh()
        scheduleProcessingTask()
        
        return super.application(application, didFinishLaunchingWithOptions: launchOptions)
    }
    
    // MARK: - BGAppRefreshTask (Short, frequent updates)
    @available(iOS 13.0, *)
    private func scheduleAppRefresh() {
        let request = BGAppRefreshTaskRequest(identifier: refreshTaskIdentifier)
        request.earliestBeginDate = Date(timeIntervalSinceNow: 15 * 60) // 15 minutes
        
        do {
            try BGTaskScheduler.shared.submit(request)
        } catch {
            print("Could not schedule app refresh: \(error)")
        }
    }
    
    @available(iOS 13.0, *)
    private func handleAppRefresh(task: BGAppRefreshTask) {
        // Schedule next refresh immediately
        scheduleAppRefresh()
        
        // Create operation queue
        let queue = OperationQueue()
        queue.maxConcurrentOperationCount = 1
        
        // Define expiration handler
        task.expirationHandler = {
            // Cancel operations if taking too long
            queue.cancelAllOperations()
        }
        
        // Perform work
        let operation = BlockOperation { [weak self] in
            // Sync data with server
            self?.performBackgroundSync()
        }
        
        operation.completionBlock = {
            task.setTaskCompleted(success: !operation.isCancelled)
        }
        
        queue.addOperations([operation], waitUntilFinished: false)
    }
    
    // MARK: - BGProcessingTask (Long, maintenance work)
    @available(iOS 13.0, *)
    private func scheduleProcessingTask() {
        let request = BGProcessingTaskRequest(identifier: processingTaskIdentifier)
        request.requiresNetworkConnectivity = true
        request.requiresExternalPower = false // Can run on battery
        
        do {
            try BGTaskScheduler.shared.submit(request)
        } catch {
            print("Could not schedule processing task: \(error)")
        }
    }
    
    @available(iOS 13.0, *)
    private func handleProcessingTask(task: BGProcessingTask) {
        scheduleProcessingTask() // Schedule next
        
        task.expirationHandler = {
            // Cleanup if expired
        }
        
        // Heavy work: Database cleanup, ML model updates, etc.
        DispatchQueue.global(qos: .background).async { [weak self] in
            self?.performMaintenance()
            
            DispatchQueue.main.async {
                task.setTaskCompleted(success: true)
            }
        }
    }
    
    // MARK: - Legacy Background Fetch (iOS < 13)
    override func application(
        _ application: UIApplication,
        performFetchWithCompletionHandler completionHandler: @escaping (UIBackgroundFetchResult) -> Void
    ) {
        // Check for new data
        let hasNewData = performBackgroundSync()
        completionHandler(hasNewData ? .newData : .noData)
    }
    
    // MARK: - Silent Push Notifications
    override func application(
        _ application: UIApplication,
        didReceiveRemoteNotification userInfo: [AnyHashable : Any],
        fetchCompletionHandler completionHandler: @escaping (UIBackgroundFetchResult) -> Void
    ) {
        // Handle silent push (content-available: 1)
        if let aps = userInfo["aps"] as? [String: Any],
           aps["content-available"] as? Int == 1 {
            
            // Perform background work
            performBackgroundSync()
            completionHandler(.newData)
        } else {
            completionHandler(.noData)
        }
    }
    
    // MARK: - Helper Methods
    private func performBackgroundSync() -> Bool {
        // Access Flutter engine in background
        // Note: UI operations are not allowed, but platform channels work
        let engine = (UIApplication.shared.delegate as? AppDelegate)?.flutterEngine
        
        // Use method channel to notify Flutter (if needed)
        // Be careful: Flutter UI is not visible in background
        
        return true // Return true if new data fetched
    }
    
    private func performMaintenance() {
        // Database cleanup, cache eviction, etc.
    }
}
```

**Explanation:**

- **BGAppRefreshTask**: Short tasks (30 seconds max) for content updates. iOS schedules these intelligently based on user behavior patterns (e.g., when user usually opens the app).
- **BGProcessingTask**: Longer tasks (minutes) for maintenance. Can require charging state (`requiresExternalPower = true`) to preserve battery.
- **Expiration Handler**: Called when system needs to suspend your app. Must cancel operations and call `setTaskCompleted` quickly to avoid termination.
- **Silent Push**: Most reliable background trigger. Send FCM/APNs with `content-available: 1`. iOS wakes your app even if force-quit (unless user disabled background refresh).
- **Flutter in Background**: You can call platform channels in background, but Flutter's UI thread is paused. Avoid UI updates; focus on data syncing.

---

## **Chapter Summary**

In this chapter, we covered native iOS integration patterns for Flutter:

### **Key Takeaways:**

1. **FlutterEngine Management**: Pre-warm engines in `AppDelegate` using `flutterEngine.run()` before `GeneratedPluginRegistrant.register()` to reduce cold start time. Share engines across `FlutterViewController` instances or create dedicated engines per screen.

2. **View Controller Embedding**: Use `addChild()` and `didMove(toParent:)` containment APIs when embedding Flutter in native iOS UIs. Set proper autoresizing masks for rotation support.

3. **Swift Concurrency**: Bridge async/await to Flutter's callback-based channels using `Task`. Always dispatch results to `MainActor` before calling `FlutterResult`. Use `Task.detached` for CPU-intensive work.

4. **Privacy Compliance**: Info.plist must contain `NS*UsageDescription` strings for camera, photos, location, microphone, etc. App Store rejects submissions missing these. Use specific, user-friendly explanations (e.g., "to scan QR codes" not "app needs camera").

5. **Universal Links**: Implement `application(_:continue:restorationHandler:)` for HTTPS deep links. Host AASA file at `/.well-known/apple-app-site-association`. Handle security-scoped resources properly when receiving files.

6. **Background Execution**: Use `BGTaskScheduler` for iOS 13+ (App Refresh and Processing tasks). Use Silent Push Notifications (`content-available`) for reliable background wakes. iOS is stricter than Android—tasks must complete quickly (30s for refresh).

### **Memory Management:**

- Use `[weak self]` in all closure handlers to prevent retain cycles between Swift and Flutter objects
- Respond to `applicationDidReceiveMemoryWarning` by clearing caches
- iOS terminates apps using excessive memory in background; save state frequently

### **Security:**

- Disable arbitrary loads in App Transport Security (ATS) for production
- Use security-scoped resources (`startAccessingSecurityScopedResource`) when accessing files shared from other apps
- Validate Universal Links using `webpageURL` to prevent spoofing

### **Next Steps:**

The next chapter (Chapter 26) will cover **Desktop & Web Support**, including:
- Windows, macOS, and Linux desktop setup
- Responsive design for desktop layouts
- Keyboard shortcuts and mouse interactions
- Web-specific considerations (CORS, PWA, CanvasKit vs HTML renderer)
- Plugin compatibility across desktop platforms

---

**End of Chapter 25**